<a href="https://colab.research.google.com/github/Aditya-singh32/My_projects/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Let's import the dataset by the help of sklearn library.

In [1]:
import numpy as np
from sklearn import datasets

# Load the Iris dataset
iris = datasets.load_iris()
iris_data = iris.data
iris_target = iris.target

> Below we have split the datasets into two halves. One half is for training dataset and it contains the rows with index 0.2.4.... from the iris dataset. While the test dataset contains the rows with index 1.3.5.... from the iris dataset. Moreover, we are not including the 3rd feature from the iris dataset into any of our dataset. The concept that helped us to achieve the task is called slicing.

Since, the iris dataset have 150 rows.
Therefore, train dataset and test dataset have 75 rows each.

In [2]:
X_train = iris_data[::2,[0, 1, 3]]
X_test = iris_data[1::2,[0,1,3]]

In [3]:
y_train = iris.target_names[iris_target[::2]]
y_test = iris.target_names[iris_target[1::2]]


In [4]:
iris_data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [5]:
print(X_train.shape,
y_train.shape)

(75, 3) (75,)


Let's concatenate our X_train along with y_train so that we could use it for X->y mapping (supervised learning). In machine learning our model learns the pattern between X and y during the training stage. This helps it later on to make predictions on new datasets.

> We are also saving these final training and testing datasets as csv files so that we could use them on weka.

In [6]:
train_matrix = np.concatenate((X_train, y_train.reshape(-1,1)), axis = 1)
train_matrix = np.concatenate((np.array(['sepal length (cm)', 'sepal width (cm)','petal width (cm)','class']).reshape(1,-1), train_matrix), axis = 0)
np.savetxt('train_data_and_labels.csv', train_matrix, delimiter = ',', fmt="%s")


In [7]:
test_matrix = np.concatenate((X_test, y_test.reshape(-1,1)), axis = 1)
test_matrix = np.concatenate((np.array(['sepal length (cm)', 'sepal width (cm)','petal width (cm)','class']).reshape(1,-1), test_matrix), axis = 0)
np.savetxt('test_data_and_labels.csv', test_matrix, delimiter = ',', fmt="%s")


## KNN model from SK-Learn Library

The results from this model should be same as the Weka results as it does the same thing that Weka does.

In [8]:
from sklearn.neighbors import KNeighborsClassifier

# I made sure to have n_neighbors as 1 and metric as euclidean so that my solutions matches with the weka solution.
model = KNeighborsClassifier(n_neighbors=1, metric='euclidean')

model.fit(X_train,y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=1)

In [9]:
# This tells us the accuracy of this model which is same as the weka and our knn function.
model.score(X_test,y_test)

0.9333333333333333

In [10]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)

# Compare predictions with ground truth
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

# Identify rows where predictions are incorrect
incorrect_rows = X_test[y_test != y_pred]

Model Accuracy: 0.9333333333333333


In [11]:
# Same as Weka's and our function
incorrect_rows

array([[6.1, 2.8, 1.3],
       [6.1, 2.8, 1.2],
       [6. , 2.2, 1.5],
       [7.2, 3. , 1.6],
       [6.3, 2.8, 1.5]])

Let's find the exact index of the rows that were predicted incorrectly. The answer is same as the Weka's answer and our KNN function.

In [12]:
np.where(y_test != y_pred)

(array([35, 36, 59, 64, 66]),)

## Our KNN function and its accuracy

In [13]:
# Function to calculate the distance between each row of two datasets
def calculate_distances(train_data, test_data):
    distances = np.zeros((test_data.shape[0], train_data.shape[0]))

    for i, test_row in enumerate(test_data):
        for j, train_row in enumerate(train_data):
            # Euclidean distance
#            distances[i, j] = np.linalg.norm(test_row - train_row)
            distances[i, j] = np.sqrt(np.sum((test_row - train_row)**2))

    return distances

Two ways to calculate the nearest neighbors of our test dataset.

In [54]:
distances = calculate_distances(X_train, X_test)

for i in range(len(X_train)):
    # Calculate distances from the i-th sample to all samples in the training set
    distances[i][i] = np.inf
    nearest_neighbors_indices[i] = np.argmin(distances[i])

nearest_neighbors_indices

array([17, 15,  8,  0, 17, 12, 19,  8, 20, 23, 22, 13, 17, 14,  1, 10,  7,
       15,  2,  0,  4, 13, 15,  1, 17, 28, 40, 48, 49, 45, 39, 39, 43, 41,
       40, 67, 67, 29, 43, 46, 45, 39, 35, 34, 45, 39, 49, 48, 37, 48, 71,
       63, 61, 65, 60, 64, 71, 52, 60, 34, 71, 63, 51, 69, 26, 61, 27, 61,
       58, 56, 60, 60, 70, 58, 69])

In [56]:
n_samples = len(X_train)

nearest_neighbors_indices = np.zeros(n_samples, dtype=int)

for i in range(n_samples):
    # Calculate distances from the i-th sample to all samples in the training set
    distances = calculate_distances(X_train, X_test[i].reshape(1, -1)).flatten()
    distances[i] = np.inf

    nearest_neighbors_indices[i] = np.argmin(distances)

nearest_neighbors_indices

array([17, 15,  8,  0, 17, 12, 19,  8, 20, 23, 22, 13, 17, 14,  1, 10,  7,
       15,  2,  0,  4, 13, 15,  1, 17, 28, 40, 48, 49, 45, 39, 39, 43, 41,
       40, 67, 67, 29, 43, 46, 45, 39, 35, 34, 45, 39, 49, 48, 37, 48, 71,
       63, 61, 65, 60, 64, 71, 52, 60, 34, 71, 63, 51, 69, 26, 61, 27, 61,
       58, 56, 60, 60, 70, 58, 69])

In [62]:
preds = y_train[nearest_neighbors_indices]

In [65]:
incorrect_rows = X_test[y_test != preds]

In [66]:
incorrect_rows

array([[6.1, 2.8, 1.3],
       [6.1, 2.8, 1.2],
       [6. , 2.2, 1.5],
       [7.2, 3. , 1.6],
       [6.3, 2.8, 1.5]])

Accuracy of our model

In [67]:
(y_test == preds).mean()

0.9333333333333333

Index of the incorrectly predicted rows. This is same as the KNN model of Weka and Sklearn.

In [68]:
np.where(y_test != preds)

(array([35, 36, 59, 64, 66]),)

## Result of Weka (Same as the KNN modle of sklearn library and our KNN function)

=== Run information ===

Scheme:       weka.classifiers.lazy.IBk -K 1 -W 0 -A "weka.core.neighboursearch.LinearNNSearch -A \"weka.core.EuclideanDistance -D -R first-last\""
Relation:     train_data_and_labels (2)
Instances:    75
Attributes:   4
              sepal length (cm)
              sepal width (cm)
              petal width (cm)
              class
Test mode:    user supplied test set:  size unknown (reading incrementally)

=== Classifier model (full training set) ===

IB1 instance-based classifier
using 1 nearest neighbour(s) for classification


Time taken to build model: 0 seconds

=== Predictions on test set ===

    inst#     actual  predicted error prediction
        1   1:setosa   1:setosa       0.974
        2   1:setosa   1:setosa       0.974
        3   1:setosa   1:setosa       0.974
        4   1:setosa   1:setosa       0.974
        5   1:setosa   1:setosa       0.974
        6   1:setosa   1:setosa       0.974
        7   1:setosa   1:setosa       0.974
        8   1:setosa   1:setosa       0.974
        9   1:setosa   1:setosa       0.974
       10   1:setosa   1:setosa       0.974
       11   1:setosa   1:setosa       0.974
       12   1:setosa   1:setosa       0.974
       13   1:setosa   1:setosa       0.974
       14   1:setosa   1:setosa       0.974
       15   1:setosa   1:setosa       0.974
       16   1:setosa   1:setosa       0.974
       17   1:setosa   1:setosa       0.974
       18   1:setosa   1:setosa       0.974
       19   1:setosa   1:setosa       0.974
       20   1:setosa   1:setosa       0.974
       21   1:setosa   1:setosa       0.974
       22   1:setosa   1:setosa       0.974
       23   1:setosa   1:setosa       0.974
       24   1:setosa   1:setosa       0.974
       25   1:setosa   1:setosa       0.974
       26 2:versicolor 2:versicolor       0.974
       27 2:versicolor 2:versicolor       0.974
       28 2:versicolor 2:versicolor       0.974
       29 2:versicolor 2:versicolor       0.974
       30 2:versicolor 2:versicolor       0.974
       31 2:versicolor 2:versicolor       0.974
       32 2:versicolor 2:versicolor       0.974
       33 2:versicolor 2:versicolor       0.974
       34 2:versicolor 2:versicolor       0.974
       35 2:versicolor 2:versicolor       0.974
       36 2:versicolor 3:virginica   +   0.974
       37 2:versicolor 3:virginica   +   0.974
       38 2:versicolor 2:versicolor       0.974
       39 2:versicolor 2:versicolor       0.974
       40 2:versicolor 2:versicolor       0.974
       41 2:versicolor 2:versicolor       0.974
       42 2:versicolor 2:versicolor       0.974
       43 2:versicolor 2:versicolor       0.974
       44 2:versicolor 2:versicolor       0.974
       45 2:versicolor 2:versicolor       0.974
       46 2:versicolor 2:versicolor       0.974
       47 2:versicolor 2:versicolor       0.974
       48 2:versicolor 2:versicolor       0.974
       49 2:versicolor 2:versicolor       0.974
       50 2:versicolor 2:versicolor       0.974
       51 3:virginica 3:virginica       0.974
       52 3:virginica 3:virginica       0.974
       53 3:virginica 3:virginica       0.974
       54 3:virginica 3:virginica       0.974
       55 3:virginica 3:virginica       0.974
       56 3:virginica 3:virginica       0.974
       57 3:virginica 3:virginica       0.974
       58 3:virginica 3:virginica       0.974
       59 3:virginica 3:virginica       0.974
       60 3:virginica 2:versicolor   +   0.974
       61 3:virginica 3:virginica       0.974
       62 3:virginica 3:virginica       0.974
       63 3:virginica 3:virginica       0.974
       64 3:virginica 3:virginica       0.974
       65 3:virginica 2:versicolor   +   0.974
       66 3:virginica 3:virginica       0.974
       67 3:virginica 2:versicolor   +   0.974
       68 3:virginica 3:virginica       0.974
       69 3:virginica 3:virginica       0.974
       70 3:virginica 3:virginica       0.974
       71 3:virginica 3:virginica       0.974
       72 3:virginica 3:virginica       0.974
       73 3:virginica 3:virginica       0.974
       74 3:virginica 3:virginica       0.974
       75 3:virginica 3:virginica       0.974

=== Evaluation on test set ===

Time taken to test model on supplied test set: 0.07 seconds

=== Summary ===

Correctly Classified Instances          70               93.3333 %
Incorrectly Classified Instances         5                6.6667 %
Kappa statistic                          0.9   
Mean absolute error                      0.0598
Root mean squared error                  0.2075
Relative absolute error                 13.4615 %
Root relative squared error             44.0212 %
Total Number of Instances               75     

=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 1.000    0.000    1.000      1.000    1.000      1.000    1.000     1.000     setosa
                 0.920    0.060    0.885      0.920    0.902      0.852    0.930     0.841     versicolor
                 0.880    0.040    0.917      0.880    0.898      0.849    0.920     0.847     virginica
Weighted Avg.    0.933    0.033    0.934      0.933    0.933      0.900    0.950     0.896     

=== Confusion Matrix ===

  a  b  c   <-- classified as
 25  0  0 |  a = setosa
  0 23  2 |  b = versicolor
  0  3 22 |  c = virginica



## Conclusion

All the 3 methods gives the same level of accuracy and answers (as expected). Therefore, everything is working fine.